In [333]:
import numpy as np
from typing import List, Tuple
from math import gcd
import random
import logging

def modinv(x,p):
    # Computes the moduler inversion of x ** p-2 mod p,
    # for p prime
    return pow(x,p-2,p)

In [334]:

# Sieve of Eratosthenes
# Code by David Eppstein, UC Irvine, 28 Feb 2002
# http://code.activestate.com/recipes/117119/

def primes_generator():
    """ Generate an infinite sequence of prime numbers.
    """
    # Maps composites to primes witnessing their compositeness.
    # This is memory efficient, as the sieve is not "run forward"
    # indefinitely, but only as long as required by the current
    # number being tested.
    #
    D = {}
    
    # The running integer that's checked for primeness
    q = 2
    
    while True:
        if q not in D:
            # q is a new prime.
            # Yield it and mark its first multiple that isn't
            # already marked in previous iterations
            # 
            yield q
            D[q * q] = [q]
        else:
            # q is composite. D[q] is the list of primes that
            # divide it. Since we've reached q, we no longer
            # need it in the map, but we'll mark the next 
            # multiples of its witnesses to prepare for larger
            # numbers
            # 
            for p in D[q]:
                D.setdefault(p + q, []).append(p)
            del D[q]
        
        q += 1

In [335]:
def generate_primes(n: int):
    primes = []
    i = 0
    for p in primes_generator():
        i += 1
        primes.append(p)
        if i == n:
            return primes

In [222]:
BIG_PRIMES_CACHE_10k = generate_primes(100000)[90000:]

In [223]:
BIG_PRIMES_CACHE_10k[0]

1159531

## Задание 1

Реализовать алгоритмы генерации ключей шифровки и дешифровки. Протестировать их для шифрования и дешифрования произвольных сообщений.

In [325]:
def generate_keys(primes_db: List[int] = BIG_PRIMES_CACHE_10k) -> Tuple[Tuple[int], Tuple[int]]:
    p, q = np.random.choice(primes_db, size = 2, replace = False)
    p, q = int(p), int(q)
    n = q*p
    print( f"Primes selected: p={p}, q={q}.\np*q = {n}")
    
    phi = (p-1)*(q-1)
    print( f"phi = {phi}")
    
    while True:
        e = random.randint(1, n)
        if gcd(e, phi) == 1:
            break
    print( f"e = {e}")
    
    d = modinv(e, phi)
    assert (d*e)%phi == 1
    print( f"d = {phi}")
    
    public_key = (e, n)
    priate_key = (d, n)

    return public_key, priate_key

In [326]:
generate_keys()

Primes selected: p=1160443, q=1248001.
p*q = 1448234024443
phi = 1448231616000
e = 165081005497
d = 1448231616000


((165081005497, 1448234024443), (1261148765833, 1448234024443))

## Проверим

In [327]:
import time

In [328]:
def encrypt(meassage: List[int], pub_key: Tuple[int]) -> List[int]:
    e, n = pub_key
    return [pow(m, e, n) for m in message]
     
def decrypt(code: List[int], private_key: Tuple[int]) -> List[int]:
    d, n=private_key
    return [pow(c, d, n) for c in code]

In [330]:
pub_key, private_key = generate_keys()
message = [55, 66, 89, 78, 93, 22]
code = encrypt(message, pub_key)
mesage_restored = decrypt(code, private_key)

message == mesage_restored

Primes selected: p=1221163, q=1190249.
p*q = 1453488039587
phi = 1453485628176
e = 758098699285
d = 1453485628176


True

In [322]:
pub_key

(1164589, 36773, 1083591)

In [320]:
message

[55, 66, 89, 78, 93, 22]

In [321]:
mesage_restored

[0.22350723902114605,
 1.0325318416297313,
 1.1028371974990483,
 0.293812594890463,
 0.007349498803341019,
 0.34417728054324376]

## Задание 2

Вариант 5

### a) 

Пусть даны открытые ключи {n,e} и сообщение m. Зашифруйте используя криптосистему RSA.

In [280]:
m = 3678
e = 176561525
n = 279976601

In [283]:
pub_key = (e, n)

code = encrypt([m], pub_key)

code

[138387333]

## b)

Расшифровать полученные шифротексты используя криптосистему RSA. Сравните результаты с исходным текстом.

In [287]:
cp = 15 / 3
cp * 3 == 15

True

In [288]:
def factorize_into_primes(num: int) -> List[int]:
    """ Assume that num is a combination of exactly 2 primes"""
    for prime in primes_generator():
        co_prime = num // prime
        if co_prime * prime == num:
            return [co_prime , prime]

In [290]:
p, q = factorize_into_primes(n)

phi = (p - 1) * (q - 1)

d = modinv(e, phi)

p, q, d

(17623, 15887, 269809349)

In [291]:
private_key = (d, n)

In [292]:
decrypt(code, private_key)

[3678]

In [293]:
m

3678